# Python code to calculate and download NDVI data
This script can be used to download any and all maps available on [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog). 

To start, you will need a free Earth Explorer and Google cloud account which can be setup as directed [here](https://developers.google.com/earth-engine/guides/python_install) or alternatively, watch the youtube video embedded below.  

Once this installation is done, import `ee` and `geemap` libraries and `ee.Initialize()`

In [ ]:
#!pip install ee
#!pip install geemap

In [ ]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import rasterio
import rioxarray

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

## Steps to download rasters after initializing
 - Download a shapefile that represents the extent of the area you are downloading rasters for
 - **OR**
 - Alternatively, construct your own area from the extents
 
 - convert that to google readable format: `ee.Feature`
 - access google earth and query the satellite image you are looking for
 - reduce it to the shapefile region
 - check on google map
 - download the file or upload it to your google drive
 - check the local file if it is working

In [ ]:
gp_df = gpd.GeoDataFrame("put your shapefile of borders of countries here")
data_image = geemap.geopandas_to_ee(gp_df)
data_image.geometry().getInfo()
    #OR
ROI = [[-81,12.5],[-81,-2.5],[-51,-2.5],[-51,12.5],[-81,12.5]] # your extent here
gp_df = ee.Algorithms.GeometryConstructors.MultiPolygon(coordinates = ROI, 
                                                      crs = "EPSG:4326", 
                                                      geodesic = False, 
                                                      maxError = 0, 
                                                      evenOdd = True)
type(gp_df)

In [ ]:
image_left = ee.ImageCollection("MODIS/061/MOD13A3").filterBounds(gp_df).filterDate('2000-01-01', '2000-12-31')
ndvi_ven = image_left.select('NDVI').median()


In [ ]:
ndvi_clipped = ndvi_ven.clip(gp_df)

In [ ]:
display_ven = {
    "min":0,
    "max":10000,
    "palette":['blue','yellow','green','darkgreen','black']
}

Map = geemap.Map()
Map.setCenter(-66,7,5)
Map.addLayer(ndvi_ven, display_ven)

Map

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=ndvi_clipped,
    description='NDVI_2020',
    folder='new_env_code',
    region=roi,
    scale = 2500,
    maxPixels= 1e13,
    #crs ='ESPG:4326',
)
task.start()

In [ ]:
ndvi_rast = rioxarray.open_rasterio("NDVI_demo.tif")
ndvi_dat = ndvi_rast.to_dataframe("ndvi_dat").reset_index()
ndvi_dat[-ndvi_dat.ndvi_dat].head()

## For other rasters
Repeat the steps above and replace the `ee.ImageCollection("MODIS/061/MOD13A3")` with the following

In [ ]:
dem = ee.Image("CGIAR/SRTM90_V4").select("elevation")
dem_clipped = dem.clip(gp_df)
task = ee.batch.Export.image.toDrive(
    image=dem_clipped,
    description='dem_srtm_demo',
    folder='new_env_code',
    region=geometry,
    #scale = 30000,
    maxPixels= 1e13,
    #crs ='ESPG:4326',
)
task.start()